In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader     # Dataloader wraps interable
from torchvision import datasets            # Datasets sorers the samples and lables
from torchvision.transforms import ToTensor # Torchvision consists of image transformatoins

# tprchvision.datasets module contains "Dataset", which has many real-world vision data
# Transform: samples
# targert_transform: labels

In [2]:
# theses are downloaded data

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

In [3]:
# DataLoader: for batching, sampling, shuffling
# batch_size=64, each element in the dataloader iterable will /
# return a batch of 64 features and labels

In [4]:
batch_size = 64

#create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N,C,H,W]: {X.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break


shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [5]:
device = torch.accelerator.current_accelerator().type \
if torch.accelerator.is_available() else "cpu"
print(device)

# mps is apple's framework for GPU

mps


In [6]:
#type(device)
#device = 'cpu'

In [7]:
#define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), #28 is pixel, input is 28*28, then output is 512
            nn.ReLU(),             #(power of 2, good for matrix)
            nn.Linear(512,512),    # mid layer, input and out features are same 512 neurals
            nn.ReLU(),
            nn.Linear(512, 10)     #10 means lable, there are total 10 different things
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
type(model)

__main__.NeuralNetwork

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size =len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device),   y.to(device)

        #compute predictoin error
        pred = model(X)         # run model
        loss = loss_fn(pred, y) # calculate the loss

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:    # print every 100th batch 
            loss, current = loss.item(),  (batch +1) * len(X)
            print(f"loss: {loss:>4f} [{current:>3d}/{size:>5d}]") # control the output format

In [11]:
def test(dataloader, model, loss_fn):  # against the test dataset
    size = len(dataloader.dataset)     # size of test data, dataset contains all data
    num_batches = len(dataloader)      # dataloader is a wrap, way to iterate the dataset
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)            # run the model
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>5f} \n")
            

In [12]:
# accuracy is easy for human to understand the model ability
# but not good for NN to training, 
# for example one output is [0,1,0,0], it is not as good as [0.45, 0.75, 0.34...]
# since entroy is continus, it is easy for traing(change the parameters of weights and biases)
# so entroy for nn, accuracy for human

import time
start = time.time()
time.sleep(4)
end = time.time()
dur = (end - start )   # 1e3 means to milliseconds
print(dur)

In [13]:
epochs = 5
import time
t_st = time.time()
for t in range(epochs):
    print(f"Epoch P{t+1}\n------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test (test_dataloader,  model, loss_fn)
print('Done!')
t_end = time.time()
t_dur = t_end - t_st
print(f'{device} use {t_dur} s')

Epoch P1
------------------
loss: 2.303349 [ 64/60000]
loss: 2.285089 [6464/60000]
loss: 2.258076 [12864/60000]
loss: 2.241733 [19264/60000]
loss: 2.240124 [25664/60000]
loss: 2.204571 [32064/60000]
loss: 2.201521 [38464/60000]
loss: 2.167345 [44864/60000]
loss: 2.154682 [51264/60000]
loss: 2.121833 [57664/60000]
Test Error: 
 Accuracy: 54.3%, Avg loss: 2.114667 

Epoch P2
------------------
loss: 2.127701 [ 64/60000]
loss: 2.118083 [6464/60000]
loss: 2.048244 [12864/60000]
loss: 2.057536 [19264/60000]
loss: 2.003985 [25664/60000]
loss: 1.944930 [32064/60000]
loss: 1.963095 [38464/60000]
loss: 1.880038 [44864/60000]
loss: 1.879958 [51264/60000]
loss: 1.803958 [57664/60000]
Test Error: 
 Accuracy: 55.7%, Avg loss: 1.803341 

Epoch P3
------------------
loss: 1.843033 [ 64/60000]
loss: 1.814256 [6464/60000]
loss: 1.694271 [12864/60000]
loss: 1.732127 [19264/60000]
loss: 1.617712 [25664/60000]
loss: 1.592913 [32064/60000]
loss: 1.604391 [38464/60000]
loss: 1.516220 [44864/60000]
loss: 1.5

Test Error: 
 Accuracy: 71.8%, Avg loss: 0.778622 

Done!
cpu use 14.480978965759277 s

In [14]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.path")

Saved PyTorch Model State to model.path


In [15]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [17]:
print(y)

9
